In [1]:
import json
import re
import numpy as np
# from __future__ import division

def read_state(State_File):
    '''
    :param State_File: file includes state set and state transition matrix
    :return N: number of states
    :return state_set: a dict contains all states' ID and name
    :return transition_prob: a dict contains transition probability 
    :return state_prob: a dict contains states and their probability
    '''
    with open(State_File, 'r') as file:
        N = int(file.readline().strip('\n'))     # read the first line to get N value
        state_set = dict()                       # store the set of state
        transition_prob = dict()                 # store transition probability  
        state_prob = dict()                      # store state initialising probability
        ID = 0                                   # ID of states
        cnt = 0                                  # number of transitions
        
        # Scan descriptive name of the states.
        while ID < N:
            state = file.readline().strip('\n').rstrip()  # one state in each line
            state_set[state] = ID
            ID = ID + 1
        
        # Scan the frequency of transitions.
        while True:
            line = file.readline()
            if not line:
                break
            items = line.split(' ')
            # Add new probability with key + value.
            transition_prob.setdefault(int(items[0]),{})[int(items[1])] = int(items[2])
            cnt = cnt + 1
        
        # Convert frequency into probability.
        for keys,values in transition_prob.items():
            total = 0
            for value in values.values():
                total = total + value
            # Scan each state in state_set.
            for state in state_set.values():
                # Case-I: state is already existing
                if state in values.keys():
#                     transition_prob[keys][state] = round((transition_prob[keys][state]+1)/(total+N-1),1)
                    transition_prob[keys][state] = (transition_prob[keys][state]+1)/(total+N-1)
                # Case-II: state is not existing
                else:
                    if state == state_set['BEGIN']:
                        transition_prob.setdefault(keys,{})[state] = 0.0
                    else:
#                         transition_prob.setdefault(keys,{})[state] = round(1/(total+N-1),1)
                        transition_prob.setdefault(keys,{})[state] = 1/(total+N-1)
            
        # Initialize state probability and Add "END" state with no outing states.
        for state in state_set.values():
            transition_prob.setdefault(state_set['END'],{})[state] = 0.0
#             state_prob[state] = round(1/N,1)
            state_prob[state] = 1/N
            
    return N, state_set, transition_prob, state_prob

def read_symbol(Symbol_File, state_set):
    '''
    :param Symbol_File: file includes symbol set and emission probability
    :param state_set: a set of state
    :return M: number of symbol
    :return symbol_set: a dict contains all symbols' ID and name
    :return emission_prob: a dict contains emission probability 
    '''
    with open(Symbol_File, 'r') as file:
        M = int(file.readline().strip('\n'))     # read the first line to get M value
        symbol_set = dict()                      # store the set of symbol
        emission_prob = dict()                   # store emission probability        
        ID = 0                                   # ID of symbols
        
        # Scan descriptive name of the symbols.
        while ID < M:
            symbol = file.readline().strip('\n').rstrip()  # one symbol in each line
#             symbol_set[ID] = symbol
            symbol_set[symbol] = ID
            ID = ID + 1
        
        # Scan the frequency of emissions.
        while True:
            line = file.readline()
            if not line:
                break
            items = line.split(' ')
            # Add new probability with key + value.
            emission_prob.setdefault(int(items[0]),{})[int(items[1])] = int(items[2])
        
        # Convert frequency into probability.
        for keys,values in emission_prob.items():
            total = 0
            for value in values.values():
                total = total + value
            # Scan each symbol in symbol_set.
            for symbol in symbol_set.values():
                # Case-I: symbol is already existing
                if symbol in values.keys():
#                     emission_prob[keys][symbol] = round((emission_prob[keys][symbol]+1)/(total+M+1),1)
                    emission_prob[keys][symbol] = (emission_prob[keys][symbol]+1)/(total+M+1)
                # Case-II: symbol is not existing
                else:
#                     emission_prob.setdefault(keys,{})[symbol] = round(1/(total+M+1),1)
                    emission_prob.setdefault(keys,{})[symbol] = 1/(total+M+1)
            # Add special symbol "UNK".
#             emission_prob.setdefault(keys,{})[M] = round(1/(total+M+1),1)
            emission_prob.setdefault(keys,{})[M] = 1/(total+M+1)
                                      
    return M, symbol_set, emission_prob

def parse_query(line):
    '''
    :param line: an address to be parsed
    :return tokens: parsed tokens sequence
    '''
    pattern = re.compile(r"[A-Za-z0-9.]+|[,&-/()]")
    tokens = pattern.findall(line)
    return tokens

In [2]:
def viterbi(O, Q, PI, A, B):
    '''
    :param O: observations
    :param Q: states
    :param PI: state probability
    :param A: transition probability
    :param B: emission probability
    :return path: the most possible state path
    :return prob: the largest probability  
    '''
    # Step 0: Define two matrix -- delta, psi.
    N = len(Q)
    T = len(O)
    # delta -- delta[t,i] -- 在时刻t，以状态i作为途径状态的最大的概率值是多少
    # delta[t,i] -- k个最高的概率值 == > delta[t,i,k]
    delta = np.zeros((T,N), float)     # highest probability of any path that ends at i
    # psi[t,i] -- 在时刻t，上述delta最大值的时候返回的状态是什么
    psi = np.zeros((T,N), int)         # argmax state
        
    # Step 1: Initialize local states when t=0.
    delta[0, :] = PI * B[:,O[0]]
    
    # 对应课件里的初始化工作
    for i in range(N):
        delta[0,i] = PI[i]*B[i,O[0]]

    # Step 2: Continue DP to compute local state in t = 1,3,...,T-1.
    for t in range(1, T):
        # Consider each state s2 (t) from previous state s1 (t-1)
        # t时刻，在状态s2确定的条件下，
        for s2 in range(N):
            # 遍历一次所有的状态，这些状态s1被认为是在t-1时间的结果
            for s1 in range(N):
                # 更新的过程 -- 对应课件里面的递归公式
                prob = delta[t-1, s1] * A[s1,s2] * B[s2,O[t]]
                if prob > delta[t, s2]:
                    delta[t, s2] = prob   # 记录最大概率值
                    psi[t, s2] = s1       #记录最大概率对应的状态值
    
    # Step 3: Compute the max delta value at T, which is the probability of most possible state sequence.
    # 直接计算最大的概率值作为返回信息
    max_prob = np.max(delta[T-1,:])
    
    # Step 4: Compute the most possible state at T.
    # 对应的状态值是哪个
    state_last = np.argmax(delta[T-1,:])
    
    # Step 5: Backtracking for t = T-1, T-2, ..., 1.
    path = np.zeros(T, int)         # initialize blank path
    path[-1] = state_last           # path is from tail to head
    
    for t in range(T - 2, -1, -1):
        # 在t+1时刻产生的最大的概率值对应的状态
        path[t] = psi[[t + 1], path[t + 1]]
    
    return path, np.log(max_prob)

In [10]:
def viterbi_algorithm(State_File, Symbol_File, Query_File): # do not change the heading of the function
    '''
    :param State_File: state file
    :param Symbol_File: symbol file
    :param Query_File: query file
    '''
    
    # Generate state information.
    # N--有多少个状态
    # state_set -- 状态集合 
    # transition_prob -- 转移矩阵
    # state_prob -- 初始状态概率值 π (暂时假定状态均匀分布)
    N, state_set, transition_prob, state_prob = read_state(State_File)
    
    # Generate symbol information.    
    # M -- 有多少个观测值
    # symbol_set -- 观测值集合
    # emission_prob -- 状态释放观测值的矩阵
    M, symbol_set, emission_prob = read_symbol(Symbol_File, state_set)
    
    # Starting query.
    with open(Query_File, 'r') as file:
        while True:
            # Parse each line.
            line = file.readline()
            if not line:
                break
            query_seq = parse_query(line)      
            
            # Generate observations and initialized state probabiltiy.
            O = [M for i in range(len(query_seq))]
            for i in range(len(query_seq)):
                if query_seq[i] in symbol_set.keys():
                    O[i] = symbol_set[query_seq[i]]

            Q = range(N)                # 观测序列
            
            # Convert dict into matrix -- A and B.
            A = np.zeros((N,N))         # 转移矩阵
            B = np.zeros((N, M+1))      # 状态释放观测值的概率矩阵
            PI = [0 for i in range(N)]  # 初始化的状态分布(暂时假定均匀分布)

            for i in range(N):
                for j in range(N):
                    A[i,j] = transition_prob[i][j]

            for i in range(N):
                for j in range(M+1):
                    if i < N-2:
                        B[i,j] = emission_prob[i][j]
                    else:
                        B[i,j] = 0.0
                        
            for i in range(N):
                PI[i] = state_prob[i]  
            
#             PI = [1/3, 1/3, 1/3, 0.0, 0.0]
#             PI = [11/36, 11/36, 11/36, 3/36, 0.0]
            path, max_pro = viterbi(O, Q, PI, A, B)
            
            
            # Join "BEGIN" and "END".
            output = []
            output.append(state_set['BEGIN'])
            output.extend(path)
            output.append(state_set['END'])
            output.append(max_pro)
            print(output)

In [11]:
State_File ='./toy_example/State_File'
Symbol_File='./toy_example/Symbol_File'
Query_File ='./toy_example/Query_File'
viterbi_result = viterbi_algorithm(State_File, Symbol_File, Query_File)

[3, 0, 0, 1, 2, 4, -8.185175305144405]
[3, 2, 1, 2, 4, -7.738888202515985]
